In [1]:
import numpy as np
from torch import nn
from torch.utils.data import DataLoader, sampler
from torch.utils.tensorboard import SummaryWriter
import torch
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
log = False

In [2]:
log = True
writer = SummaryWriter('./log')

In [3]:
from src.utils.data import getPandas, writePandas
data = getPandas('data')
test = data.sample(frac=0.2, random_state=10)
data = data.drop(test.index)
validate = data.sample(frac=0.1, random_state=10)
train = data.drop(validate.index)
test = test.reset_index(drop=True)
train = train.reset_index(drop=True)
validate = validate.reset_index(drop=True)
writePandas('data_test', test)
writePandas('data_train', train)
writePandas('data_validate', validate)

In [3]:
from src.dl.loader import RegressDataset
from src.dl.resnet import RegressCNN
from sklearn.model_selection import KFold
import collections
train_set = RegressDataset('train')
val_set = RegressDataset('validate')
test_set = RegressDataset('test')
fold_num = 5
kf = KFold(n_splits=fold_num, shuffle=True, random_state=10)
best_models = np.empty(dtype=collections.OrderedDict, shape=fold_num)
net = RegressCNN().cuda()
loss_fn = nn.SmoothL1Loss().cuda()
#lr = 1e-2
#optim = torch.optim.Adam(net.parameters(), lr=lr)
lr = 1e-2
#optim = torch.optim.SGD([{'params': net.fc.weight, 'lr': 1e-2}], lr=lr)
optim = torch.optim.SGD(net.parameters(), lr=lr)
epoch = 100
train_loader = DataLoader(train_set, batch_size=8)
val_loader = DataLoader(val_set, batch_size=8)
for i in range(epoch):
    total_loss = 0
    predy = np.array([])
    y = np.array([])
    net.train()
    for step, [img, labels, score] in enumerate(train_loader):
        img = img.cuda()
        labels = labels.cuda()
        score = score.cuda()
        output = net(img, labels).squeeze(-1)
        loss = loss_fn(output, score)

        optim.zero_grad()
        loss.backward()
        optim.step()
        #scheduler.step()
        
        output = output.cpu().detach().numpy()
        score = score.cpu().detach().numpy()
        predy = np.concatenate((predy, output), axis=0)
        y = np.concatenate((y, score), axis=0)
        total_loss += loss.item()
        
    nmse = np.sum((y - predy) ** 2) / np.sum(y ** 2)
    mse = np.sum((y - predy) ** 2) / len(train_set)
    rmse = np.sqrt(mse)
    mae = np.sum(np.abs(y - predy)) / len(train_set)
    meany = np.mean(y)
    rsquared = r2_score(y, predy)
    print('train epoch {}, loss {}, nmse {}, rmse {}, mae {}, rsquared {}'.format(i, total_loss, nmse, rmse, mae, rsquared))
    
    if log:
        writer.add_scalar('train/Loss', total_loss, i)
        writer.add_scalar('train/NMSE', nmse, i)
        writer.add_scalar('train/RMSE', rmse, i)
        writer.add_scalar('train/MAE', mae, i)
        writer.add_scalar('train/rsquared', rsquared, i)
        
    net.eval()
    total_loss = 0
    predy = np.array([])
    y = np.array([])
    with torch.no_grad():
        total = 0
        correct = 0
        for step, [img, labels, score] in enumerate(val_loader):
            img = img.cuda()
            labels = labels.cuda()
            score = score.cuda()
            output = net(img, labels).squeeze(-1)
            loss = loss_fn(output, score)

            output = output.cpu().detach().numpy()
            score = score.cpu().detach().numpy()
            #print("y: ", score)
            #print("y_pred: ", output)
            predy = np.concatenate((predy, output), axis=0)
            y = np.concatenate((y, score), axis=0)
            total_loss += loss.item()

    nmse = np.sum((y - predy) ** 2) / np.sum(y ** 2)
    mse = np.sum((y - predy) ** 2) / len(val_set)
    rmse = np.sqrt(mse)
    mae = np.sum(np.abs(y - predy)) / len(val_set)
    meany = np.mean(y)
    rsquared = r2_score(y, predy)
    print('validation epoch {}, loss {}, nmse {}, rmse {}, mae {}, rsquared {}'.format(i, total_loss, nmse, rmse, mae, rsquared))
    if log:
        writer.add_scalar('validation/Loss', total_loss, i)
        writer.add_scalar('validation/NMSE', nmse, i)
        writer.add_scalar('validation/RMSE', rmse, i)
        writer.add_scalar('validation/MAE', mae, i)
        writer.add_scalar('validation/rsquared', rsquared, i)

train epoch 0, loss 1.238169008374675, nmse 0.5576014266935511, rmse 0.3136014795915315, mae 0.24791561923659725, rsquared -0.8507326480077531
validation epoch 0, loss 0.06367563467945775, nmse 0.42387602993288237, rmse 0.2105593529209412, mae 0.16465632709149858, rsquared -0.15558957783938787
train epoch 1, loss 0.7165192718651713, nmse 0.31615727153171413, rmse 0.23613901772643772, mae 0.19001760296363374, rsquared -0.04935632571537463
validation epoch 1, loss 0.07204838595883484, nmse 0.4797253264331429, rmse 0.22400176059708146, mae 0.17174906849421132, rsquared -0.3078483997774488
train epoch 2, loss 0.7155078450978667, nmse 0.3157359133697811, rmse 0.2359816082891661, mae 0.18984858964080026, rsquared -0.04795779753832474
validation epoch 2, loss 0.07229845382065427, nmse 0.4813739661114916, rmse 0.22438633634146174, mae 0.17194984944943925, rsquared -0.3123429941762257
train epoch 3, loss 0.7146349724735706, nmse 0.31539373258584397, rmse 0.23585370034766948, mae 0.1897471190314

In [3]:
from src.dl.loader import BlockRegressDataset
from src.dl.resnet import RegressResNet3d
from sklearn.model_selection import KFold
import collections
train_set = BlockRegressDataset('train')
val_set = BlockRegressDataset('validate')
test_set = BlockRegressDataset('test')
fold_num = 5
kf = KFold(n_splits=fold_num, shuffle=True, random_state=10)
best_models = np.empty(dtype=collections.OrderedDict, shape=fold_num)
net = RegressResNet3d().cuda()
loss_fn = nn.SmoothL1Loss().cuda()
#lr = 1e-2
#optim = torch.optim.Adam(net.parameters(), lr=lr)
lr = 1e-3
#optim = torch.optim.SGD([{'params': net.fc.weight, 'lr': 1e-2}], lr=lr)
optim = torch.optim.SGD(net.parameters(), lr=lr)
epoch = 100
train_loader = DataLoader(train_set, batch_size=8)
val_loader = DataLoader(val_set, batch_size=8)
for i in range(epoch):
    total_loss = 0
    predy = np.array([])
    y = np.array([])
    net.train()
    for step, [img, labels, score] in enumerate(train_loader):
        img = img.cuda()
        labels = labels.cuda()
        score = score.cuda()
        output = net(img, labels).squeeze(-1)
        loss = loss_fn(output, score)

        optim.zero_grad()
        loss.backward()
        optim.step()
        #scheduler.step()
        
        output = output.cpu().detach().numpy()
        score = score.cpu().detach().numpy()
        predy = np.concatenate((predy, output), axis=0)
        y = np.concatenate((y, score), axis=0)
        total_loss += loss.item()
        
    nmse = np.sum((y - predy) ** 2) / np.sum(y ** 2)
    mse = np.sum((y - predy) ** 2) / len(train_set)
    rmse = np.sqrt(mse)
    mae = np.sum(np.abs(y - predy)) / len(train_set)
    meany = np.mean(y)
    rsquared = r2_score(y, predy)
    print('train epoch {}, loss {}, nmse {}, rmse {}, mae {}, rsquared {}'.format(i, total_loss, nmse, rmse, mae, rsquared))
    
    if log:
        writer.add_scalar('train/Loss', total_loss, i)
        writer.add_scalar('train/NMSE', nmse, i)
        writer.add_scalar('train/RMSE', rmse, i)
        writer.add_scalar('train/MAE', mae, i)
        writer.add_scalar('train/rsquared', rsquared, i)
        
    net.eval()
    total_loss = 0
    predy = np.array([])
    y = np.array([])
    with torch.no_grad():
        total = 0
        correct = 0
        for step, [img, labels, score] in enumerate(val_loader):
            img = img.cuda()
            labels = labels.cuda()
            score = score.cuda()
            output = net(img, labels).squeeze(-1)
            loss = loss_fn(output, score)

            output = output.cpu().detach().numpy()
            score = score.cpu().detach().numpy()
            #print("y: ", score)
            #print("y_pred: ", output)
            predy = np.concatenate((predy, output), axis=0)
            y = np.concatenate((y, score), axis=0)
            total_loss += loss.item()

    nmse = np.sum((y - predy) ** 2) / np.sum(y ** 2)
    mse = np.sum((y - predy) ** 2) / len(val_set)
    rmse = np.sqrt(mse)
    mae = np.sum(np.abs(y - predy)) / len(val_set)
    meany = np.mean(y)
    rsquared = r2_score(y, predy)
    print('validation epoch {}, loss {}, nmse {}, rmse {}, mae {}, rsquared {}'.format(i, total_loss, nmse, rmse, mae, rsquared))
    if log:
        writer.add_scalar('validation/Loss', total_loss, i)
        writer.add_scalar('validation/NMSE', nmse, i)
        writer.add_scalar('validation/RMSE', rmse, i)
        writer.add_scalar('validation/MAE', mae, i)
        writer.add_scalar('validation/rsquared', rsquared, i)

train epoch 0, loss 76.2948952867059, nmse 0.2923725867191226, rmse 0.22708293039888186, mae 0.18030249206669766, rsquared 0.029587325785412788
validation epoch 0, loss 7.459754966934131, nmse 0.42479690567412537, rmse 0.21078795018615362, mae 0.16226482330006475, rsquared -0.15810011000898005
train epoch 1, loss 66.04962458012086, nmse 0.2531131950811844, rmse 0.21128738004466602, mae 0.16673977112354485, rsquared 0.15989301434167025
validation epoch 1, loss 7.446343205243306, nmse 0.4239673417145707, rmse 0.21058203114873628, mae 0.16533574936384027, rsquared -0.1558385160567004
train epoch 2, loss 56.31746606768795, nmse 0.21581676944158154, rmse 0.19510066898222367, mae 0.15286047245792922, rsquared 0.2836834303642999
validation epoch 2, loss 7.671067939525478, nmse 0.4366770406693776, rmse 0.2137151390018426, mae 0.1714066501234379, rsquared -0.19048825940731318
train epoch 3, loss 46.38304410493647, nmse 0.17774252926918788, rmse 0.17705648690164505, mae 0.13793395655235102, rsqu

In [5]:
test_loader = DataLoader(test_set, batch_size=64)
net.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for step, [img, labels, score] in enumerate(test_loader):
        img = img.cuda()
        labels = labels.cuda()
        score = score.cuda()
        #score = torch.Tensor(np.random.randint(0, 3, score.size())).type(torch.LongTensor).cuda()
        output = net(img, labels).squeeze(-1)
        loss = loss_fn(output, score)
        
        predy = np.concatenate((predy, output.cpu().detach().numpy()), axis=0)
        y = np.concatenate((y, score.cpu().detach().numpy()), axis=0)
        total_loss += loss.item()
nmse = np.sum((y - predy) ** 2) / np.sum(y ** 2)
mse = np.sum((y - predy) ** 2) / len(test_set)
rmse = np.sqrt(mse)
mae = np.sum(np.abs(y - predy)) / len(test_set)
meany = np.mean(y)
rsquared = 1 - np.sum((predy - y) ** 2) / np.sum((y - meany) ** 2)
print('epoch {}, loss {}, nmse {}, rmse {}, mae {}, rsquared {}'.format(i, total_loss, nmse, rmse, mae, rsquared))

epoch 99, loss 14.81796445565536, nmse 0.38246806504802155, rmse 0.2794655588970451, mae 0.2638362386154233, rsquared -0.5403551487274227


In [ ]:
loss_fn = nn.SmoothL1Loss().cuda()
lr = 1e-3
optim = torch.optim.SGD([
    {'params': net.fc.weight, 'lr': 1e-2}
    ], lr=lr, momentum=0.3)
#optim = torch.optim.SGD(net.parameters(), lr=1e-1, momentum=0.3)
#scheduler = torch.optim.lr_scheduler.ExponentialLR(optim, gamma=0.99)
#optim = torch.optim.Adam(net.parameters(), lr=lr, betas=[0.3, 0.1])
epoch = 100
dataset = BlockRegressDataset('train')
kf = KFold(n_splits=5, shuffle=True, random_state=10)
for fold, (train_idx, val_idx) in enumerate(kf.split(np.arange(len(dataset)))):
    train_sampler = sampler.SubsetRandomSampler(train_idx)
    val_sampler = sampler.SubsetRandomSampler(val_idx)
    train_loader = DataLoader(dataset, batch_size=64, sampler=train_sampler)
    val_loader = DataLoader(dataset, batch_size=64, sampler=val_sampler)
    for i in range(epoch):
        total_loss = 0
        predy = np.array([])
        y = np.array([])
        net.train()
        for step, [img, labels, score] in enumerate(train_loader):
            img = img.cuda()
            labels = labels.cuda()
            score = score.cuda()
            output = net(img, labels).squeeze(-1)
            loss = loss_fn(output.float(), score.float())

            optim.zero_grad()
            loss.backward()
            optim.step()
            #scheduler.step()
        
        net.eval()
        with torch.no_grad():
            for step, [img, labels, score] in enumerate(val_loader):
                img = img.cuda()
                labels = labels.cuda()
                score = score.cuda()
                output = net(img, labels).squeeze(-1)
                loss = loss_fn(output.float(), score.float())
        
                predy = np.concatenate((predy, output.cpu().detach().numpy()), axis=0)
                y = np.concatenate((y, score.cpu().detach().numpy()), axis=0)

                total_loss += loss.item()

        nmse = np.sum((y - predy) ** 2) / np.sum(y ** 2)
        mse = np.sum((y - predy) ** 2) / len(train_idx)
        mae = np.sum(np.abs(y - predy)) / len(train_idx)
        meany = np.mean(y)
        rsquared = np.sum((predy - meany) ** 2) / np.sum((y - meany) ** 2)
        print('epoch {}, loss {}, nmse {}, mae {}, rsquared {}'.format(i, total_loss, nmse, mae, rsquared))
        writer.add_scalar('Loss', total_loss, i)
        writer.add_scalar('NMSE', nmse, i)
        writer.add_scalar('MAE', mae, i)
        writer.add_scalar('rsquared', rsquared, i)

writer.close()

In [ ]:
predy = []
y = []
for fold, (train_idx, val_idx) in enumerate(kf.split(np.arange(len(dataset)))):
    train_sampler = sampler.SubsetRandomSampler(train_idx)
    val_sampler = sampler.SubsetRandomSampler(val_idx)
    train_loader = DataLoader(dataset, batch_size=64, sampler=train_sampler)
    val_loader = DataLoader(dataset, batch_size=64, sampler=val_sampler)
    net.eval()
    with torch.no_grad():
        for step, [img, labels, score] in enumerate(train_loader):
            img = img.cuda()
            labels = labels.cuda()
            score = score.cuda()
            output = net(img, labels).squeeze(-1)
            loss = loss_fn(output.float(), score.float())
    
            predy = np.concatenate((predy, output.cpu().detach().numpy()), axis=0)
            y = np.concatenate((y, score.cpu().detach().numpy()), axis=0)

            total_loss += loss.item()

    nmse = np.sum((y - predy) ** 2) / np.sum(y ** 2)
    mse = np.sum((y - predy) ** 2) / len(train_idx)
    mae = np.sum(np.abs(y - predy)) / len(train_idx)
    meany = np.mean(y)
    rsquared = np.sum((predy - meany) ** 2) / np.sum((y - meany) ** 2)
    print(rsquared)
    print(mse)
    print(mae)
    break